<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 📚 מדריך מקיף ל-PySpark: עבודה עם DataFrames

מחברת זו מכסה את הנושאים הבאים:
1. ✅ החלת פונקציות על עמודות (Apply Functions) -  [Apply Functions](https://sparkbyexamples.com/pyspark/pyspark-apply-function-to-column/)

2. 🔄 שימוש ב-Transform -  [Transform](https://sparkbyexamples.com/pyspark/pyspark-transform-function/)

3. 🎯 יצירת UDF (User Defined Functions) -  [User Defined Functions](https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/)

4. 🔗 איחוד DataFrames (Union/UnionByName) -  [Union/Union](https://sparkbyexamples.com/pyspark/pyspark-union-and-unionall/)
,  [UnionByName](https://sparkbyexamples.com/pyspark/pyspark-unionbyname/)

5. 🤝 חיבור DataFrames (Joins) -  [Joins](https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/)


---

## 🚀 הכנה והגדרת SparkSession

נתחיל עם יצירת SparkSession ו-DataFrame לדוגמה:

In [2]:
from pyspark.sql import SparkSession

# יצירת SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# יצירת DataFrame לדוגמה
columns = ["Seqno", "Name"]
data = [("1", "john jones"),
        ("2", "tracey smith"),
        ("3", "amy sanders")]

df = spark.createDataFrame(data=data, schema=columns)
df.show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+



---

## 1️⃣ החלת פונקציות על עמודות

### 🔹 שימוש ב-withColumn()

`withColumn()` היא פונקציית טרנספורמציה המאפשרת להחיל פונקציה מובנית או מותאמת אישית על עמודה.

In [3]:
from pyspark.sql.functions import upper

# החלת פונקציית upper על עמודת Name
df.withColumn("Upper_Name", upper(df.Name)).show()

+-----+------------+------------+
|Seqno|        Name|  Upper_Name|
+-----+------------+------------+
|    1|  john jones|  JOHN JONES|
|    2|tracey smith|TRACEY SMITH|
|    3| amy sanders| AMY SANDERS|
+-----+------------+------------+



### 🔹 שימוש ב-select()

`select()` משמש לבחירת עמודות, וניתן גם להחיל עליהן פונקציות:

In [4]:
# בחירת עמודות והחלת פונקציה
df.select("Seqno", "Name", upper(df.Name)).show()

+-----+------------+------------+
|Seqno|        Name| upper(Name)|
+-----+------------+------------+
|    1|  john jones|  JOHN JONES|
|    2|tracey smith|TRACEY SMITH|
|    3| amy sanders| AMY SANDERS|
+-----+------------+------------+



### 🔹 שימוש ב-SQL

ניתן גם להשתמש ב-SQL כדי להחיל פונקציות:

In [5]:
# יצירת temporary view
df.createOrReplaceTempView("TAB")

# שאילתת SQL
spark.sql("select Seqno, Name, UPPER(Name) from TAB").show()

+-----+------------+------------+
|Seqno|        Name| upper(Name)|
+-----+------------+------------+
|    1|  john jones|  JOHN JONES|
|    2|tracey smith|TRACEY SMITH|
|    3| amy sanders| AMY SANDERS|
+-----+------------+------------+



---

## 2️⃣ יצירת UDF מותאם אישית

### 📝 שלב 1: יצירת פונקציית Python

In [6]:
# פונקציה שממירה מחרוזת לאותיות גדולות
def upperCase(str):
    return str.upper()

### 📝 שלב 2: המרת הפונקציה ל-UDF

In [7]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

# יצירת UDF
upperCaseUDF = udf(lambda z: upperCase(z), StringType())

### 📝 שלב 3: שימוש ב-UDF

#### עם withColumn():

In [8]:
df.withColumn("Cureated_Name", upperCaseUDF(col("Name"))).show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated_Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



#### עם select():

In [9]:
df.select(col("Seqno"), upperCaseUDF(col("Name")).alias("Name")).show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |JOHN JONES  |
|2    |TRACEY SMITH|
|3    |AMY SANDERS |
+-----+------------+



#### עם SQL:

In [10]:
# רישום UDF לשימוש ב-SQL
spark.udf.register("upperCaseUDF", upperCaseUDF)

df.createOrReplaceTempView("TAB")
spark.sql("select Seqno, Name, upperCaseUDF(Name) from TAB").show()

+-----+------------+------------------+
|Seqno|        Name|upperCaseUDF(Name)|
+-----+------------+------------------+
|    1|  john jones|        JOHN JONES|
|    2|tracey smith|      TRACEY SMITH|
|    3| amy sanders|       AMY SANDERS|
+-----+------------+------------------+



### ⚠️ אנוטציות - דרך קצרה ליצירת UDF

In [11]:
@udf(returnType=StringType())
def upperCase(str):
    return str.upper()

df.withColumn("Cureated_Name", upperCase(col("Name"))).show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated_Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



### 🔍 טיפול ב-Null Values

**חשוב!** UDF עלול להחזיר שגיאות אם העמודה מכילה ערכי null. הנה דרך לטפל בכך:

In [12]:
# DataFrame עם null
columns = ["Seqno", "Name"]
data = [("1", "john jones"),
        ("2", "tracey smith"),
        ("3", "amy sanders"),
        ("4", None)]

df2 = spark.createDataFrame(data=data, schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")

# דרך 1: בדיקה פנימית ב-UDF
# spark.udf.register("_nullsafeUDF", upperCase, StringType())
# spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2").show(truncate=False)

# דרך 2: שימוש ב-WHERE ב-SQL
# spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " +
#           "where Name is not null and _nullsafeUDF(Name) like '%John%'").show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |NULL        |
+-----+------------+



### ⚡ ביצועים - UDF vs פונקציות מובנות

**חשוב לדעת:** UDF הוא תהליך יקר ביותר מבחינת ביצועים! כדאי תמיד להשתמש בפונקציות המובנות של PySpark SQL כשאפשר.

ה-UDF הוא "קופסה שחורה" עבור PySpark - הוא לא יכול לבצע אופטימיזציות. השתמש ב-UDF רק כאשר אין פונקציה מובנית מתאימה.

---

## 3️⃣ שימוש ב-Pandas UDF עם PySpark

ניתן להשתמש ב-Pandas API מעל PySpark:

In [13]:
import pyspark.pandas as ps
import numpy as np
import pyspark.pandas as ps
ps.set_option("compute.fail_on_ansi_mode", False)

# יצירת DataFrame עם Pandas על PySpark
psdf = ps.DataFrame(
    {'fee': [20000, 25000, 30000, 22000, np.nan],
     'Discount': [1000, 1500, 1500, 1200, 3000]}
)
print(psdf)

# הגדרת פונקציה
def add(data):
    return data[0] + data[1]

# שימוש בפונקציה
addDF = psdf.apply(add, axis=1)
print(addDF)

/Users/mosheshulman/tutorials/PySpark/.venv/lib/python3.13/site-packages/pyspark/pandas/__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
/Users/mosheshulman/tutorials/PySpark/.venv/lib/python3.13/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


       fee  Discount
0  20000.0      1000
1  25000.0      1500
2  30000.0      1500
3  22000.0      1200
4      NaN      3000
0    21000.0
1    26500.0
2    31500.0
3    23200.0
4        NaN
dtype: float64


/Users/mosheshulman/tutorials/PySpark/.venv/lib/python3.13/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/var/folders/kb/bm6sw7g57_5_708m47q7jqzm0000gn/T/ipykernel_14041/1333334986.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return data[0] + data[1]
/Users/mosheshulman/tutorials/PySpark/.venv/lib/python3.13/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


---

## 4️⃣ Transform Function

PySpark מספק שני סוגים של `transform()`:
1. **DataFrame.transform()** - זמין מ-Spark 3.0
2. **sql.functions.transform()** - עבודה עם עמודות מסוג Array

### 🔹 DataFrame.transform()

משמש לשרשור טרנספורמציות מותאמות אישית:

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# יצירת DataFrame
simpleData = (("Java", 4000, 5), 
              ("Python", 4600, 10),
              ("Scala", 4100, 15), 
              ("Scala", 4500, 15),
              ("PHP", 3000, 20))
              
columns = ["CourseName", "fee", "discount"]

df = spark.createDataFrame(data=simpleData, schema=columns)
df.printSchema()
df.show(truncate=False)

root
 |-- CourseName: string (nullable = true)
 |-- fee: long (nullable = true)
 |-- discount: long (nullable = true)

+----------+----+--------+
|CourseName|fee |discount|
+----------+----+--------+
|Java      |4000|5       |
|Python    |4600|10      |
|Scala     |4100|15      |
|Scala     |4500|15      |
|PHP       |3000|20      |
+----------+----+--------+



### יצירת פונקציות טרנספורמציה מותאמות:

In [15]:
from pyspark.sql.functions import upper

# טרנספורמציה 1: המרת שם קורס לאותיות גדולות
def to_upper_str_columns(df):
    return df.withColumn("CourseName", upper(df.CourseName))

# טרנספורמציה 2: הפחתת אחוז הנחה מהמחיר
def reduce_price(df, reduceBy):
    return df.withColumn("new_fee", df.fee - reduceBy)

# טרנספורמציה 3: חישוב מחיר לאחר הנחה
def apply_discount(df):
    return df.withColumn("discounted_fee",  
                        df.new_fee - (df.new_fee * df.discount) / 100)

### שרשור הטרנספורמציות:

In [ ]:
# שימוש ב-transform() לשרשור
df2 = df.transform(to_upper_str_columns) \
        .transform(reduce_price, 1000) \
        .transform(apply_discount)

df2.show()

### 🔹 sql.functions.transform()

משמש להחלת טרנספורמציות על עמודות מסוג ArrayType:

In [ ]:
from pyspark.sql.functions import upper, transform
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# יצירת DataFrame עם Array
data = [
    ("James", "Smith", ["Java", "Scala", "C++"]),
    ("Michael", "Rose", ["Spark", "Java", "C++"]),
    ("Robert", "Williams", ["CSharp", "VB"])
]

df = spark.createDataFrame(data=data, schema=["Name", "Languages1", "Languages2"])
df.printSchema()
df.show()

### החלת טרנספורמציה על Array:

In [ ]:
# המרת כל איברי המערך לאותיות גדולות
df.select(transform(col("Languages2"), lambda x: upper(x)).alias("languagesUP")).show()

---

## 5️⃣ איחוד DataFrames - Union

### 🔹 union() - איחוד לפי מיקום

`union()` ממזג שני DataFrames **לפי מיקום העמודות** (לא לפי שם):

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# DataFrame ראשון
simpleData = [
    ("James", "Sales", "NY", 90000, 34, 10000),
    ("Michael", "Sales", "NY", 86000, 56, 20000),
    ("Robert", "Sales", "CA", 81000, 30, 23000),
    ("Maria", "Finance", "CA", 90000, 24, 23000)
]

columns = ["employee_name", "department", "state", "salary", "age", "bonus"]
df = spark.createDataFrame(data=simpleData, schema=columns)
df.printSchema()
df.show(truncate=False)

# DataFrame שני
simpleData2 = [
    ("James", "Sales", "NY", 90000, 34, 10000),
    ("Maria", "Finance", "CA", 90000, 24, 23000),
    ("Jen", "Finance", "NY", 79000, 53, 15000),
    ("Jeff", "Marketing", "CA", 80000, 25, 18000),
    ("Kumar", "Marketing", "NY", 91000, 50, 21000)
]

columns2 = ["employee_name", "department", "state", "salary", "age", "bonus"]
df2 = spark.createDataFrame(data=simpleData2, schema=columns2)
df2.printSchema()
df2.show(truncate=False)

### ביצוע union:

In [ ]:
# union משלב את כל השורות (כולל כפילויות)
unionDF = df.union(df2)
unionDF.show(truncate=False)

### הסרת כפילויות:

In [ ]:
# distinct() מסיר שורות כפולות
disDF = df.union(df2).distinct()
disDF.show(truncate=False)

### 🔹 unionByName() - איחוד לפי שם עמודה

`unionByName()` ממזג DataFrames **לפי שם העמודות**, לא לפי המיקום:

In [ ]:
# DataFrames עם עמודות באותם שמות אך בסדר שונה
df1 = spark.createDataFrame([[5, 2, 6]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[6, 7, 3]], ["col1", "col2", "col0"])

# unionByName מסדר את העמודות לפי שם
df3 = df1.unionByName(df2)
df3.printSchema()
df3.show()

### טיפול במספר שונה של עמודות:

In [ ]:
# DataFrames עם מספר שונה של עמודות
df1 = spark.createDataFrame([[5, 2, 6]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[6, 7, 3]], ["col1", "col2", "col3"])

# allowMissingColumns=True ממלא null עבור עמודות חסרות
df3 = df1.unionByName(df2, allowMissingColumns=True)
df3.printSchema()
df3.show()

---

## 6️⃣ Joins - חיבור DataFrames

PySpark תומך בכל סוגי ה-Joins המוכרים מ-SQL:
- **Inner Join** - רק שורות תואמות
- **Left Outer Join** - כל השורות מהשמאל + תואמות מהימין
- **Right Outer Join** - כל השורות מהימין + תואמות משמאל
- **Full Outer Join** - כל השורות משני הצדדים
- **Left Semi Join** - רק שורות מהשמאל שיש להן התאמה
- **Left Anti Join** - רק שורות מהשמאל שאין להן התאמה
- **Self Join** - חיבור של טבלה לעצמה
- **Cross Join** - מכפלה קרטזית

### הכנת DataFrames לדוגמה:

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# DataFrame עובדים
emp = [
    (1, "Smith", -1, "2018", "10", "M", 3000),
    (2, "Rose", 1, "2010", "20", "M", 4000),
    (3, "Williams", 1, "2010", "10", "M", 1000),
    (4, "Jones", 2, "2005", "10", "F", 2000),
    (5, "Brown", 2, "2010", "40", "", -1),
    (6, "Brown", 2, "2010", "50", "", -1)
]

empColumns = ["emp_id", "name", "superior_emp_id", "year_joined", 
              "emp_dept_id", "gender", "salary"]

empDF = spark.createDataFrame(data=emp, schema=empColumns)
empDF.printSchema()
empDF.show(truncate=False)

# DataFrame מחלקות
dept = [
    ("Finance", 10),
    ("Marketing", 20),
    ("Sales", 30),
    ("IT", 40)
]

deptColumns = ["dept_name", "dept_id"]
deptDF = spark.createDataFrame(data=dept, schema=deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

### 🔹 Inner Join

מחזיר רק שורות שיש להן התאמה בשני ה-DataFrames:

In [ ]:
# Inner Join
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "inner").show(truncate=False)

### 🔹 Left Outer Join

מחזיר את כל השורות מהשמאל, ושורות תואמות מהימין (null אם אין התאמה):

In [ ]:
# Left Outer Join
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "left").show(truncate=False)

# ניתן גם להשתמש ב-"leftouter"
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "leftouter").show(truncate=False)

### 🔹 Right Outer Join

מחזיר את כל השורות מהימין, ושורות תואמות מהשמאל (null אם אין התאמה):

In [ ]:
# Right Outer Join
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "right").show(truncate=False)

# ניתן גם להשתמש ב-"rightouter"
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "rightouter").show(truncate=False)

### 🔹 Full Outer Join

מחזיר את כל השורות משני ה-DataFrames (null במקומות שאין התאמה):

In [ ]:
# Full Outer Join
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "outer").show(truncate=False)

# ניתן גם להשתמש ב-"full" או "fullouter"
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "full").show(truncate=False)
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "fullouter").show(truncate=False)

### 🔹 Left Semi Join

מחזיר רק שורות מהשמאל שיש להן התאמה בימין (ללא עמודות מהימין):

In [ ]:
# Left Semi Join
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "leftsemi").show(truncate=False)

### 🔹 Left Anti Join

מחזיר רק שורות מהשמאל שאין להן התאמה בימין:

In [ ]:
# Left Anti Join
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, "leftanti").show(truncate=False)

### 🔹 Self Join

חיבור של DataFrame לעצמו (שימושי למצוא קשרים היררכיים):

In [ ]:
# Self Join - מציאת מנהל של כל עובד
empDF.alias("emp1").join(
    empDF.alias("emp2"),
    col("emp1.superior_emp_id") == col("emp2.emp_id"),
    "inner"
).select(
    col("emp1.emp_id"),
    col("emp1.name"),
    col("emp2.emp_id").alias("superior_emp_id"),
    col("emp2.name").alias("superior_emp_name")
).show(truncate=False)

### 🔹 שימוש ב-SQL Expressions

In [ ]:
# יצירת temporary views
empDF.createOrReplaceTempView("EMP")
deptDF.createOrReplaceTempView("DEPT")

# שימוש ב-SQL
joinDF = spark.sql(
    "select * from EMP e, DEPT d where e.emp_dept_id == d.dept_id"
).show(truncate=False)

# Inner Join עם SQL
joinDF2 = spark.sql(
    "select * from EMP e INNER JOIN DEPT d ON e.emp_dept_id == d.dept_id"
).show(truncate=False)

---

## 📊 דוגמה מקיפה - שילוב כל הנושאים

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, upper, udf
from pyspark.sql.types import StringType

# יצירת Spark session
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# יצירת DataFrames
columns = ["Seqno", "Name"]
data = [("1", "john jones"),
        ("2", "tracey smith"),
        ("3", "amy sanders")]

df = spark.createDataFrame(data=data, schema=columns)
df.show(truncate=False)

# שימוש בפונקציה מובנית
df.withColumn("Upper_Name", upper(df.Name)).show()

# שימוש ב-select
df.select("Seqno", "Name", upper(df.Name)).show()

# שימוש ב-SQL
df.createOrReplaceTempView("TAB")
spark.sql("select Seqno, Name, UPPER(Name) from TAB").show()

# יצירת פונקציה מותאמת
def upperCase(str):
    return str.upper()

# המרה ל-UDF
upperCaseUDF = udf(lambda z: upperCase(z), StringType())

# שימוש ב-UDF עם withColumn
df.withColumn("Cureated_Name", upperCaseUDF(col("Name"))).show(truncate=False)

# שימוש ב-UDF עם select
df.select(col("Seqno"), upperCaseUDF(col("Name")).alias("Name")).show(truncate=False)

# רישום UDF ושימוש ב-SQL
spark.udf.register("upperCaseUDF", upperCaseUDF)
df.createOrReplaceTempView("TAB")
spark.sql("select Seqno, Name, upperCaseUDF(Name) from TAB").show()

---

## 📚 משאבים נוספים

### קישורים שימושיים:
- [PySpark SQL Functions Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)
- [PySpark UDF Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.udf.html)
- [Spark By Examples - sparkbyexamples.com](https://sparkbyexamples.com/pyspark/)

### נושאים קשורים:
- PySpark Window Functions
- PySpark Aggregate Functions
- PySpark Date and Timestamp Functions
- PySpark Broadcasting
- PySpark Partitioning

---

## 💡 טיפים חשובים

1. **ביצועים**: תמיד העדף פונקציות מובנות על פני UDF
2. **Null Handling**: תמיד טפל בערכי null ב-UDF
3. **Transform**: השתמש ב-transform() לשרשור פעולות בצורה קריאה
4. **Join Type**: בחר את סוג ה-Join המתאים לצרכים שלך
5. **unionByName**: השתמש ב-unionByName() כשיש לך עמודות באותם שמים אך בסדר שונה

---

**הערה**: מחברת זו נוצרה על בסיס המדריכים מ-SparkByExamples.com